# qiskit_alt

This python package provides a thin wrapper around some features of Qiskit that have been (re-)implemented in Julia and provides a Python interface. The input and output of the Python interface are the same as the input and output to Python qiskit. At present, we have prepared two high level demonstrations

* Performing the Jordan-Wigner transform from a Fermionic operator to a Pauli operator.

* Computing the Fermionic operator from integrals computed by `pyscf`.

In both cases, we will see that the `qiskit_alt` implementation is much more performant.

We have also prepared some lower-level demonstrations of performance gains

* Converting an operator from the computational basis to the Pauli basis.

* Creating a `SparsePauliOp` from a list of strings

The Python package has been installed in a virtual environment created with `python -m venv ./env`. The Julia packages have been installed in a local environment in the standard way, via a spec in `Project.toml` file.

When we import the package `qiskit_alt`, the Julia environment is also activated.

In [1]:
import qiskit_alt  # The message below is from Julia

┌ Warning: Package ElectronicStructure does not have PyCall in its dependencies:
│ - If you have ElectronicStructure checked out for development and have
│   added PyCall as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ElectronicStructure
└ Loading PyCall into ElectronicStructure from project dependency, future warnings for ElectronicStructure are suppressed.
  Activating project at `~/myrepos/quantum_repos/qiskit_alt`


We assume that no one is familiar with Julia, much less with `pyjulia`, the package we use to call Julia from Python. So, we inject a bit of tutorial.

The default `Module` in Julia `Main` is available. You can think of it as a namespace. And, as always, objects from the `Module` `Base` have been imported into `Main`.

As an example of how `pyjulia` works, we create an `Array` of `Float64` zeros on the Julia side, and they are automatically copied to a numpy array when returned to Python.

In [2]:
qiskit_alt.Main.zeros(10)

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall ~/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

There are several ways to call Julia from Python and vice versa, and to specifiy features such as the copying vs. sharing semantics. We won't go into any of this in this demo.

## Electronic structure

Part of a workflow for, say, VQE involves using qiskit-nature to do the following:
* Get a description of a model Hamiltonian from the package `pyscf` by passing it a description of the geometry of a molecule.
* Convert that description of a Hamiltonian to a qiskit-native Fermionic operator.
* Convert the Fermionic operator to a qubit operator expressed in the Pauli basis.

The last step above may be done in several ways, one of which is known as the Jordan-Wigner transform. It is this step that we will benchmark here.

### qiskit-nature

First, we see how this is done in qiskit-nature. We need to specify the geometry of the molecule and the
[basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)). We choose `sto3g`, one of the smallest, simplest, basis sets.

In [3]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# Specify the geometry of the H_2 molecule
geometry = [['H', [0., 0., 0.]],
            ['H', [0., 0., 0.735]]]

basis = 'sto3g'

Then, we compute the fermionic Hamiltonian like this.

In [4]:
molecule = Molecule(geometry=geometry,
                     charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF)

from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

fermionic_hamiltonian = second_q_op[0]

The Jordan-Wigner transform is performed like this.

In [5]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
nature_qubit_op = qubit_converter.convert(fermionic_hamiltonian)
nature_qubit_op.primitive

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.81054798+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### qiskit_alt

The only high-level code in `qiskit_alt` was written to support this demo. So doing the JW-transform is less verbose.

In [6]:
fermi_op = qiskit_alt.fermionic_hamiltonian(geometry, basis)
pauli_op = qiskit_alt.jordan_wigner(fermi_op)
pauli_op.simplify() # The Julia Pauli operators use a different sorting convention; we sort again for comparison. 

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.09057899+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### Benchmarking

Computing the Hamiltonian for a larger molecule or a larger basis set takes more time and produces a Hamiltonian with more factors and terms. Here we compare the performance of `qiskit_alt` and `qiskit-nature` on combinations of $\text{H}_2$ and $\text{H}_2\text{O}$ molecules for several basis sets.

First we benchmark qiskit-nature, and record the times in `nature_times`.

In [7]:
%run ../bench/jordan_wigner_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 7.57 ms
geometry=h2_geometry, basis='631g' 106.81 ms
geometry=h2_geometry, basis='631++g' 555.08 ms
geometry=h2o_geometry, basis='sto3g' 652.11 ms
geometry=h2o_geometry, basis='631g' 18862.67 ms


[7.573255989700556,
 106.81066708639264,
 555.0834431312978,
 652.1143527701497,
 18862.674361094832]

Next we benchmark qiskit_alt, and record the times in `alt_times`.

In [8]:
%run ../bench/jordan_wigner_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 0.83 ms
geometry=h2_geometry, basis='631g' 3.78 ms
geometry=h2_geometry, basis='631++g' 21.65 ms
geometry=h2o_geometry, basis='sto3g' 18.90 ms
geometry=h2o_geometry, basis='631g' 587.82 ms


[0.8277131244540215,
 3.7832845002412796,
 21.645490266382694,
 18.90419856645167,
 587.8180948086083]

We compare the relative performance.

In [9]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[9.149614481099414,
 28.23225878983745,
 25.64429986570413,
 34.49574180454411,
 32.08930539512646]

We see that
* qiskit_alt is at least ten times faster
* The relative performance increases as the problem in some sense gets larger.

In fact, another problem, not shown here, finishes in 18s with qiskit_alt and in 5730s in qiskit-nature.
In this case, `qiskit_alt` is 320 times faster than `qiskit-nature`. I don't have an idea about the origin of this scaling.

### Computing the Fermonic operator

Computing the Fermionic operator from the output of `pyscf` is also much more efficient in `qiskit_alt`.

We benchmark qiskit-nature computing the fermionic Hamiltonian

In [10]:
%run ../bench/fermionic_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 58.57 ms
geometry=h2_geometry, basis='631g' 99.65 ms
geometry=h2_geometry, basis='631++g' 288.70 ms
geometry=h2o_geometry, basis='sto3g' 441.23 ms
geometry=h2o_geometry, basis='631g' 24174.41 ms


[58.57091718353331,
 99.64704178273678,
 288.6968629434705,
 441.2329154089093,
 24174.414460081607]

We benchmark qiskit_alt computing the fermionic Hamiltonian.

In [11]:
%run ../bench/fermionic_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 58.55 ms
geometry=h2_geometry, basis='631g' 61.24 ms
geometry=h2_geometry, basis='631++g' 64.39 ms
geometry=h2o_geometry, basis='sto3g' 76.69 ms
geometry=h2o_geometry, basis='631g' 207.81 ms


[58.552867313846946,
 61.23726116493344,
 64.38925410620868,
 76.69081739149988,
 207.8069806098938]

We compare the relative performance.

In [12]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[1.00030826619625,
 1.6272289107501445,
 4.483618686858389,
 5.753399564858647,
 116.33109912444708]

We see again that, as the problem size increases, `qiskit_alt` is increasingly more performant.

## Discussion

The Julia implemenation consists of these packages

* [`QuantumOps.jl`](https://github.com/jlapeyre/QuantumOps.jl) implementing Fermionic and Pauli operators and calculations using them.

* [`ElectronicStructure.jl`](https://github.com/jlapeyre/ElectronicStructure.jl) provides an interface to electronic structure packages. Only one is supported, `pyscf`.

* [`QiskitQuantumInfo.jl`](https://github.com/jlapeyre/QiskitQuantumInfo.jl) provides data structures that mirror Python Qiskit data structures. These are used as intermedidate structures for converting from `QuantumOps` and `ElectronicStructure` to Python Qiskit. In the future these might be used directly for calculations.


The Python interface is a Python package `qiskit_alt`. This could contain a mixture of Julia and Python code. Or all the Julia code might be moved to the Julia packages.

### Implementation

In the examples above, the following happens.

* Julia code calls `pyscf` and stores the results in Julia data structures.

* These data are used to construct a Fermionic operator as a data structure defined in `QuantumOps`.

* The Jordan-Wigner transform, implemented in `QuantumOps` is used to compute a Pauli operator.

* The Pauli operator (as a structure in `QuantumOps`) is converted to a Qiskit-like operator defined in `QiskitQuantumInfo.jl`.

* The operator defined in `QiskitQuantumInfo.jl` is sent to Python and converted to numpy arrays, which are then used to construct native Qiskit types. The conversion to numpy arrays is provided by `pyjulia`.

### Complexity, dynamism

* It is worth noting that operators in `QuantumOps` are *not* highly optimized implementations. In fact, much of the code for the two types of operators is shared, they inherit from a parent class. There are other implementations of Pauli operators in Julia that are much more efficient for instance in [`QuantumClifford.jl`](https://github.com/Krastanov/QuantumClifford.jl).

* This performance does not come at the cost of complexity, as with C++. Many people, including myself, find Julia to be more dynamic and expressive than Python. It is easier and faster to put ideas into code. The limitations of the language are less obtrusive.

# More demos

Here are some smaller scale demonstrations.

## Converting a matrix to the Pauli basis

Here we convert a matrix representing an operator in the computational basis to the Pauli basis.
In this case, `qiskit_alt` is much more performant than `qiskit.quantum_info`.
This is how it is done in `QuantumOps`.

In [13]:
from qiskit_alt import QuantumOps, PauliSum_to_SparsePauliOp
import numpy as np
m = np.random.rand(2**3, 2**3) # 3-qubit operator
pauli_sum = QuantumOps.PauliSum(m) # This is a wrapped Julia object
PauliSum_to_SparsePauliOp(pauli_sum) # Convert to qiskit.quantum_info.SparsePauliOp

SparsePauliOp(['III', 'XII', 'YII', 'ZII', 'IXI', 'XXI', 'YXI', 'ZXI', 'IYI', 'XYI', 'YYI', 'ZYI', 'IZI', 'XZI', 'YZI', 'ZZI', 'IIX', 'XIX', 'YIX', 'ZIX', 'IXX', 'XXX', 'YXX', 'ZXX', 'IYX', 'XYX', 'YYX', 'ZYX', 'IZX', 'XZX', 'YZX', 'ZZX', 'IIY', 'XIY', 'YIY', 'ZIY', 'IXY', 'XXY', 'YXY', 'ZXY', 'IYY', 'XYY', 'YYY', 'ZYY', 'IZY', 'XZY', 'YZY', 'ZZY', 'IIZ', 'XIZ', 'YIZ', 'ZIZ', 'IXZ', 'XXZ', 'YXZ', 'ZXZ', 'IYZ', 'XYZ', 'YYZ', 'ZYZ', 'IZZ', 'XZZ', 'YZZ', 'ZZZ'],
              coeffs=[ 0.65552691+0.j        ,  0.52010932+0.j        ,  0.        -0.09852002j,
  0.08057372+0.j        ,  0.57781268+0.j        ,  0.60462426+0.j        ,
  0.        +0.02749087j, -0.02477894+0.j        ,  0.        -0.02715969j,
  0.        -0.04693143j, -0.05295258-0.j        ,  0.        -0.14257379j,
 -0.05205437+0.j        , -0.10590266+0.j        ,  0.        -0.00317546j,
  0.10803236+0.j        ,  0.40295782+0.j        ,  0.50360106+0.j        ,
  0.        +0.17727659j,  0.0599451 +0.j        ,  0.41185

Note that the `numpy` matrix was automatically converted to a Julia `Matrix`.

We run benchmarks of conversion of matrices to the Pauli basis.

In [14]:
%run ../bench/from_matrix_quantum_info.py

nqubits=2, 1.17 ms
nqubits=3, 4.82 ms
nqubits=4, 23.68 ms
nqubits=5, 136.67 ms
nqubits=6, 1000.70 ms
nqubits=7, 10787.20 ms
nqubits=8, 195281.88 ms


In [15]:
%run ../bench/from_matrix_alt.py

nqubits=2, 0.70 ms
nqubits=3, 0.81 ms
nqubits=4, 1.32 ms
nqubits=5, 10.43 ms
nqubits=6, 21.95 ms
nqubits=7, 105.00 ms
nqubits=8, 670.40 ms


Here are the ratios of the times for `qiskit.quantum_info` to those for `qiskit_alt`.

In [16]:
[t_pyqk / t_qk_alt for t_pyqk, t_qk_alt in zip(pyqk_times, qk_alt_times)]

[1.6635590360827133,
 5.933566433365912,
 17.93891004225413,
 13.102248020782996,
 45.595398465153316,
 102.73387784902454,
 291.29186896388705]

Again, the performance gain increases with the problem size.

## Creating a `SparsePauliOp` from a list of strings


Here, we create a `SparsePauliOp` from a list of `n` strings, each with `k` single-Pauli factors, and simplify the result.

First, using `qiskit.quantum_info`

In [17]:
%run ../bench/pauli_from_list_qinfo.py

k=10, n=10, 0.00033735299948602917 sec
k=10, n=100, 0.0015016289660707116 sec
k=10, n=1000, 0.013455566647462548 sec
k=10, n=5000, 0.06510934496764094 sec
k=10, n=10000, 0.13238832913339138 sec
k=10, n=100000, 1.3394438498653471 sec
k=100, n=10, 0.0003386484459042549 sec
k=100, n=100, 0.001628452562727034 sec
k=100, n=1000, 0.0144160759402439 sec
k=100, n=5000, 0.07109922040253877 sec
k=100, n=10000, 0.14616359397768974 sec
k=100, n=100000, 1.5016626939177513 sec


Now, using `qiskit_alt`

In [18]:
%run ../bench/pauli_from_list_alt.py

k=10, n=10, 0.005888417130336165 sec
k=10, n=100, 0.0007687784032896161 sec
k=10, n=1000, 0.001907213544473052 sec
k=10, n=5000, 0.007400881499052048 sec
k=10, n=10000, 0.02045779600739479 sec
k=10, n=100000, 0.1939062522025779 sec
k=100, n=10, 0.0007018823409453035 sec
k=100, n=100, 0.0008772603468969464 sec
k=100, n=1000, 0.00559198809787631 sec
k=100, n=5000, 0.013534089992754161 sec
k=100, n=10000, 0.028353752102702856 sec
k=100, n=100000, 0.32433034824207424 sec


The results were written to lists `quantum_info_times` and `qkalt_times`. We compare the performance:

In [19]:
[x / y for x,y in zip(quantum_info_times, qkalt_times)]

[0.057290947977860045,
 1.9532663243988322,
 7.055091804719861,
 8.79751215797477,
 6.471289922215353,
 6.907687785466568,
 0.482486060908954,
 1.8562933666011598,
 2.577987593664362,
 5.253343256961026,
 5.1550000665963545,
 4.630040642379041]

We see that the performance improvement in `qiskit_alt` is significant, but does not increase with the number of terms `n`. Further benchmarks show that the time required to convert the strings from Python to Julia is about three times longer than the time required to construct the operators. 